Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">logic</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">sets</a>.<a class="ProveItLink" href="../../theory.ipynb">equivalence</a>.<a class="ProveItLink" href="../../theorems.ipynb#set_not_equiv_existential">set_not_equiv_existential</a> theorem
========

In [ ]:
import proveit
from proveit import defaults, n, x, y, A, B, P, Q, S, Lambda
from proveit.numbers import one
from proveit.logic.sets.equivalence import set_equiv_fold, unfold_set_not_equiv
theory = proveit.Theory() # the theorem's theory

In [ ]:
%proving set_not_equiv_existential

In [ ]:
defaults.assumptions = set_not_equiv_existential.conditions

We proceed using a proof by contradiction, showing that $\forall_{x}((x \in A)=(x \in B))$ (i.e. the negation of $\exists_{x}((x\in A)\ne(x \in B))$) leads to a contradiction of the condition that $A \not\cong B$.

In [ ]:
from proveit.logic import SetNotEquiv
A_notequiv_B = SetNotEquiv(A, B)

In [ ]:
not_A_equiv_B = A_notequiv_B.unfold()

In [ ]:
from proveit.logic import SetEquiv
A_equiv_B = SetEquiv(A, B)

In [ ]:
from proveit import x, A, B
from proveit.logic import Equals, Forall, InSet
forall_x_then_A_equiv_B = A_equiv_B.conclude(assumptions=[Forall(x, Equals(InSet(x, A), InSet(x, B)))])

In [ ]:
forall_x_then_A_equiv_B

In [ ]:
contra = not_A_equiv_B.derive_contradiction(assumptions=[Forall(x, Equals(InSet(x, A), InSet(x, B)))])

In [ ]:
not_for_all_judgment = contra.deny_assumption(Forall(x, Equals(InSet(x, A), InSet(x, B))))

We are almost done. We still need the system to understand that $\forall_{x}((x \in A)=(x \in B)) = \neg(\exists_{x}((x \in A)\ne(x \in B)))$, and this doesn't seem to have been encoded in the Forall class methods. Eventually this might deserve an equality prover method of its own, perhaps called something existential_translation()?

In [ ]:
from proveit.logic.booleans.quantification.existence import not_forall_implies_exists_not
not_forall_implies_exists_not

In [ ]:
not_for_all_expr = not_for_all_judgment.expr

In [ ]:
_x = _y = not_for_all_expr.operand.instance_params

In [ ]:
_n = _x.num_elements()

In [ ]:
_Q = Lambda(_x, not_for_all_expr.operand.conditions)

In [ ]:
_P = Lambda(_x, not_for_all_expr.operand.instance_expr)

In [ ]:
# A temporary work-around for an empty conjunction-related error when we instantiate
# in the next cell. If we manually execute this step, we avoid an error in the instantiation below
from proveit.logic import And, Equals, Forall, InSet
Equals(Forall(x, Equals(InSet(x, A), InSet(x, B)), conditions=[And()]),
       Forall(x, Equals(InSet(x, A), InSet(x, B)))).prove()

In [ ]:
not_forall_implies_exists_not.instantiate({P:_P, Q:_Q, x:_x, y:_y, n:_n})

In [ ]:
%qed

### Proof Tangent (cells commented-out)

The cells below explore and document some errors that occur when the Exists.derive_negated_forall() method is called repeatedly.

#### Inadvertently discovered that a _second_ call to the Exists.derive_negated_forall() method leads to a puzzling error, where we see an empty conjunction as a condition in the quantified Forall expression and it causes an error when the system cannot or does not recognize that the version with the empty conjunction condition it equal to the version without the empty conjunction.
Temporarily leaving some of the following cells here for later continued exploration of this odd error.

Here we _manually_ go through the process for the second attempt (instead of the repeated method call) to see if we can narrow down the source of this (new-ish) error.

We begin with the theorm imported and instantiated inside the `Exists.derive_negated_forall()` method:

In [ ]:
# this is the theorem imported and instantiated inside the Exists.derive_negated_forall() method
# from proveit.logic.booleans.quantification.existence import exists_not_implies_not_forall
# exists_not_implies_not_forall

#### And the step-by-step creation of the substitutions for the instantiation:

In [ ]:
# _n = exists_expr.instance_params.num_elements()

In [ ]:
# _x = exists_expr.instance_params

In [ ]:
# _y = _x

In [ ]:
# the following creates the substitution of () for Q
# which eventually gets interpreted as an empty conjunction And()
# from proveit import Lambda
# _Q = Lambda(_x, exists_expr.conditions)

In [ ]:
# _P = Lambda(x, exists_expr.instance_expr.operand)

#### And then the actual instantiation, which works just fine:

In [ ]:
# _impl = exists_not_implies_not_forall.instantiate(
#                 {P: _P, Q: _Q, x: _x, y: _y, n: _n})

#### The `Exists.derive_negated_forall()` method then finishes by deriving the consequent of the implication:

In [ ]:
# _impl.derive_consequent(assumptions=[exists_expr])

And this continues to work no matter how many times we repeatedly call the derive_consequent() method on the impication:

In [ ]:
# _impl.derive_consequent(assumptions=[exists_expr])

In [ ]:
# _impl.derive_consequent(assumptions=[exists_expr])

#### But if we once again directly call the `derive_negated_forall()` method, we get an error, involving the consequent somehow not simplifying the empty conjunction:

In [ ]:
# exists_expr.derive_negated_forall(assumptions=[exists_expr])

But strangely enough, when we retrieve that unsatisfied condition, we find that it is readily_provable, so it's puzzling why this isn't being dealt with very easily:

In [ ]:
# from proveit import Instantiation
# error_condition = Instantiation.unsatisfied_condition

In [ ]:
# Instantiation.condition_assumptions

In [ ]:
# error_condition.readily_provable()

In [ ]:
# error_condition.lhs.conditions

In [ ]:
# from proveit.logic import Equals
# Equals(error_condition.lhs, error_condition.rhs).prove()

#### And if we run it a 3rd time, we no longer get the error but a somewhat bizarre result (part of the “bizarreness” is the fact that it no longer gives an error at all):

In [ ]:
# exists_expr.derive_negated_forall(assumptions=[exists_expr])

In [ ]:
# display(_impl.consequent.expr_info(details=True))